In [3]:
import numpy
%matplotlib notebook
from matplotlib import pyplot

In [4]:
from soapy import atmosphere, WFS, confParse
from aotools import circle
from aotools.wfs import wfslib

In [5]:
SOAPY_CONF = "conf/test_conf.py"

In [6]:
def scaleScrnR0(scrns, oldR0, newR0):
    r0Scrns = scrns.copy() * ((oldR0/newR0)**(5./6.))
    return r0Scrns

In [7]:
# Initialise the WFS
config = confParse.Configurator(SOAPY_CONF)
config.loadSimParams()

config.tel.telDiam=1.

mask = circle.circle(config.sim.pupilSize/2., config.sim.simSize)

wfs = WFS.ShackHartmann(
        config.sim, config.wfss[0], config.atmos, config.lgss[0], mask)

In [8]:
# Make a set of phase screens to test
nScrns = 20
nSub = 4
scrnSize = config.sim.simSize*nSub
simSize = config.sim.simSize

# r0 to make scrns initially
scrn_r0 = 1.
pxlScale = 1./config.sim.pupilSize
L0 = 100
l0 = 0.01

# Make all the phase screens
scrns = numpy.zeros((nScrns*nSub*nSub, simSize, simSize))
for i in range(nScrns):
    wholeScrn = atmosphere.ft_sh_phase_screen(scrn_r0, scrnSize, pxlScale, L0, l0)
    for nx in range(nSub):
        for ny in range(nSub):
            index = i*(nSub**2) + nx*nSub + ny
            scrns[index] = wholeScrn[nx*simSize:(nx+1)*simSize, ny*simSize:(ny+1)*simSize]


In [9]:
scrns *= 500./(2*numpy.pi)
print(scrns.min(), scrns.max(), scrns.var())

(-632.47027694172016, 657.09318205815362, 34448.44098455902)


In [10]:
# Get slopes for each screen, for each r0
nR0 = 10
r0s = numpy.linspace(0.05, scrn_r0, nR0)
slopes = numpy.zeros((nR0, len(scrns), wfs.activeSubaps*2))

for ir, r0 in enumerate(r0s):
    r0Scrns = scaleScrnR0(scrns, 1, r0)
    print(r0, r0Scrns.var())
    for i, s in enumerate(r0Scrns):
        slopes[ir, i] = wfs.frame(s)

(0.050000000000000003, 5076365.6666467935)
(0.15555555555555556, 765644.25529340992)
(0.26111111111111113, 322944.07224834955)
(0.36666666666666664, 183393.46440315503)
(0.47222222222222221, 120298.12850930273)
(0.57777777777777783, 85947.816589632799)
(0.68333333333333335, 64980.327345253951)
(0.78888888888888897, 51145.783968609292)
(0.89444444444444449, 41487.18365042939)
(1.0, 34448.44098455902)


In [11]:
for i in range(nR0):
    print(slopes[i].min(), slopes[i].max(), slopes[i].var())

(-2.6927259058936048, 2.5753003811104165, 0.39473732624651614)
(-1.0865418757478746, 0.97585433386014309, 0.063910132326514407)
(-0.70590145996492559, 0.62783764443427259, 0.028847898137345989)
(-0.57176385334089019, 0.49327459385609274, 0.01735126421098359)
(-0.42407082820187192, 0.41130767925554679, 0.011830427487931302)
(-0.34640760122869985, 0.33243159703588354, 0.0086652115242219045)
(-0.31423587486702864, 0.30135951638275849, 0.0066621155247113826)
(-0.28692726270269553, 0.275058772126755, 0.0053030274915836723)
(-0.26377015495189049, 0.25280299350854563, 0.0043358563679314604)
(-0.2440215661252001, 0.23385151089716949, 0.003621282362803159)


In [12]:
# Turn slopes into radians
wfsPxlScale_asec = float(config.wfss[0].subapFOV)/config.wfss[0].pxlsPerSubap
wfsPxlScale_deg = wfsPxlScale_asec/3600.
wfsPxlScale_rad = wfsPxlScale_deg * (numpy.pi/180.)
print(wfsPxlScale_rad)

6.06017101387e-07


In [13]:
# Convert slopes to rad
slopes_rad = slopes*wfsPxlScale_rad

In [14]:
# Measure r0 using slope variance
wvl = config.wfss[0].wavelength
# wvl = 600e-9
print("Wavelength (m): {}".format(wvl))
d = wfs.subapDiam
print("Subap Diameter (m): {}".format(d))
measuredR0 = numpy.zeros(nR0)
for i in range(nR0):
    sVar = slopes_rad[i].var(0).mean()
    measuredR0[i] = ((0.162*(wvl**2) * d**(-1./3)) / sVar)**(3./5)

Wavelength (m): 6e-07
Subap Diameter (m): 1.0


In [26]:
pyplot.figure()
pyplot.plot(r0s, measuredR0)
pyplot.plot(r0s, r0s, color="k", linestyle="--")
pyplot.grid()
pyplot.xlabel("Phase Screen $r_0$")
pyplot.ylabel("$r_0$ measured from WFS slopes")

<IPython.core.display.Javascript object>

ValueError: x and y must have same first dimension

In [ ]:
slopes_rad[1].var(0).mean()

In [ ]:
wfs.activeSubaps

In [27]:
nScrns = 100

# Alternative way, which uses Soapy atmosphere
config.atmos.scrnNo = 1
slopes = numpy.zeros((nR0, nScrns, wfs.activeSubaps))
for ir, r0 in enumerate(r0s):
    config.atmos.r0 = r0
    config.atmos.randomScrns = True
    atmos = atmosphere.atmos(config.sim, config.atmos)
    for i in range(nScrns):
        slopes[ir, i] = wfs.frame(atmos.moveScrns())[:wfs.activeSubaps]

In [28]:
slopes.shape

(10, 100, 52)

In [29]:
# Convert slopes to rad
slopes_rad = slopes*wfsPxlScale_rad
# Measure r0 using slope variance
wvl = config.wfss[0].wavelength
# wvl = 600e-9
print("Wavelength (m): {}".format(wvl))
d = wfs.subapDiam
print("Subap Diameter (m): {}".format(d))
measuredR0 = numpy.zeros(nR0)
for i in range(nR0):
    sVar = slopes_rad[i].var(0).mean()
    measuredR0[i] = ((0.358*(wvl**2) * d**(-1./3)) / sVar)**(3./5)

Wavelength (m): 6e-07
Subap Diameter (m): 1.0


In [30]:
pyplot.figure()
pyplot.plot(r0s, measuredR0)
pyplot.plot(r0s, r0s, color="k", linestyle="--")
pyplot.grid()
pyplot.xlabel("Phase Screen $r_0$")
pyplot.ylabel("$r_0$ measured from WFS slopes")

<IPython.core.display.Javascript object>

In [31]:
# Perhaps this is something to do with WFS linearity? Would see a change with pixel scale if so
nFovs = 5
subapFovs = numpy.linspace(3, 10, nFovs)
nScrns = 100

# Alternative way, which uses Soapy atmosphere
config.atmos.scrnNo = 1

slopes = numpy.zeros((nFovs, nR0, nScrns, wfs.activeSubaps*2))

for ifov, fov in enumerate(subapFovs):
    config.wfss[0].subapFOV = fov
    wfs = WFS.ShackHartmann(
            config.sim, config.wfss[0], config.atmos, config.lgss[0], mask)
    for ir, r0 in enumerate(r0s):
        config.atmos.r0 = r0
        config.atmos.randomScrns = True
        atmos = atmosphere.atmos(config.sim, config.atmos)
        for i in range(nScrns):
            slopes[ifov, ir, i] = wfs.frame(atmos.moveScrns())[:wfs.activeSubaps]

ValueError: could not broadcast input array from shape (52) into shape (104)

In [ ]:
wfsPxlScales_rad = []
slopes_rad = slopes.copy()
# Turn slopes into radians
for ifov, fov in enumerate(subapFovs):
    wfsPxlScale_asec = float(fov)/config.wfss[0].pxlsPerSubap
    wfsPxlScale_deg = wfsPxlScale_asec/3600.
    wfsPxlScale_rad = wfsPxlScale_deg * (numpy.pi/180.)
    
    slopes_rad[ifov] *= wfsPxlScale_rad
    print(wfsPxlScale_rad)

In [ ]:
# Measure r0 using slope variance
wvl = config.wfss[0].wavelength
# wvl = 600e-9
print("Wavelength (m): {}".format(wvl))
d = wfs.subapDiam
print("Subap Diameter (m): {}".format(d))
measuredR0 = numpy.zeros((nFovs, nR0))
for ifov, fov in enumerate(subapFovs):
    for i in range(nR0):
        sVar = slopes_rad[ifov, i].var(0).mean()
        measuredR0[ifov, i] = ((0.358*(wvl**2) * d**(-1./3)) / sVar)**(3./5)

In [ ]:
pyplot.figure()
pyplot.plot(r0s, r0s, color="k", linestyle="--", label="Expected")

pyplot.grid()
pyplot.xlabel("Phase Screen $r_0$")
pyplot.ylabel("$r_0$ measured from WFS slopes")

for ifov, fov in enumerate(subapFovs):
    pxlScale = float(fov)/config.wfss[0].pxlsPerSubap
    pyplot.plot(r0s, measuredR0[ifov], label="Pixel Scale: {:.2f}\" ".format(pxlScale))
    
pyplot.legend(loc=0)